<h1 style="color:blue;">PROJETO: Análise e detecção de fraudes em transações imobiliárias</h1>
<h2 style="color:lightblue;">Parte 2</h2>

Na parte 2, foram colocadas em práticas as tarefas para tratar dados ausentes, sabemos que isso é um problema e temos algumas variáveis faltantes que foram preenchidas com base em agrupamentos de outras features que julgamos estarem próximas, pois queremos que os dados sejam os mais próximos possíveis do real, toda decisão tomada irá influenciar nas informações geradas, cabe ao time de negócio junto com os cientistas de dados tomarem a decisão mais acertiva e caso necessário voltar e ajustar.

Após isso, a etapa é de **feature engineering** ou **engenharia de atributos**, ou seja, vamos criar outros atributos com base nas variáveis existentes, um exemplo é, temos uma feature de "largura do terreno" e outra de "profundidade do terreno", podemos criar uma terceira multiplicando essas duas que se tornará uma variável "área do terreno", se quisermos podemos até apagar as duas primeiras porque uma já as representa.

A etapa de feature engineering é muito importante, porque o intuito dela é facilitar para que o algoritmo consiga encontrar de forma mais fácil os padrões nos dados, caso exista... Importante ressaltar que não estamos criando padrões.


# Sumário

**<p style="color:lightgrey;">`Parte 1:` Análise Exploratória**</p>
<p style="color:lightgrey;">
   1. Importando as bibliotecas e carregando os dados<br>
   2. Análise Exploratória dos dados<br>
   3. Conclusão<br>
</p>

**`Parte 2:` Limpeza, transformação e Feature Engineering**
   1. Importando as bibliotecas e carregando os dados<br>
   2. Limpeza e transformação dos dados<br>
      2.1 Limpando valores ausentes da variável: ZIP<br>
      2.2 Limpando valores ausentes das variáveis: FULLVAL, AVLAND, AVTOT, BLDFRONT, BLDDEPTH, LTFRONT e LTDEPTH<br>
      2.3 Limpando valores ausentes da variável: STORIES<br>
   3. Engenharia de Atributos<br>
   4. Conclusão<br>

**<p style="color:lightgrey;">`Parte 3:` Modelagem e calculo do Score de Fraude</p>**
<p style="color:lightgrey;">
   1. Importando as bibliotecas e carregando os dados<br>
   2. Padronização das variáveis<br>
   3. Aplicando PCA<br>
      3.1. Aplicação da Escala Z em componentes PCA Padronizados<br>
      3.2 Calculando o Fraud Score 1<br>
   4. Aplicacando Deep Learning<br>
      4.1 Construção do Modelo<br>
      4.2 Avaliação do modelo<br>
      4.3 Calculando o Fraud Score 2<br>
   5. Calculando o Score Final do Score de Fraude e apresentando os Resultados<br>
   6. Conclusão<br>
   7. Referências<br>
</p>

## 1. Importando as bibliotecas e carregando os dados

In [1]:
# imports
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
%matplotlib inline

plt.style.use('ggplot')

In [2]:
# carregando os dados
df = pd.read_csv('Dados/dataset.csv')

# visualizando as primeiras linhas
df.head()

,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,EXT,STORIES,FULLVAL,AVLAND,AVTOT,EXLAND,EXTOT,EXCD1,STADDR,ZIP,EXMPTCL,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,1046,NaN,NaN,21400000.0,4225500.0,9630000.0,4225500.0,9630000.0,4600.0,1 LIBERTY ISLAND,10004.0,X3,0,0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27,0,NaN,NaN,193800000.0,14310000.0,87210000.0,14310000.0,87210000.0,4600.0,1 ELLIS ISLAND,10004.0,X3,0,0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709,564,E,3.0,104686000.0,39008700.0,47108700.0,39008700.0,47108700.0,2191.0,MARGINAL STREET,10004.0,X1,709,564,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793,551,NaN,2.0,39200000.0,15255000.0,17640000.0,15255000.0,17640000.0,2191.0,PIER 6,10004.0,X1,85,551,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323,1260,NaN,1.0,272300000.0,121050000.0,122535000.0,121050000.0,122535000.0,2231.0,BATTERY PARK,10004.0,X1,89,57,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR


In [3]:
# visualizando as últimas linhas
df.tail()

,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,EXT,STORIES,FULLVAL,AVLAND,AVTOT,EXLAND,EXTOT,EXCD1,STADDR,ZIP,EXMPTCL,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE
1070989,1070990,5080500083,5,8050,83,NaN,"TOBIN, GALE",A1,1,60,120,NaN,2.5,438000.0,14536.0,23322.0,1620.0,1620.0,1017.0,142 BENTLEY STREET,10307.0,NaN,30,26,NaN,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR
1070990,1070991,5080500086,5,8050,86,NaN,SHERRI MILINAZZO,A1,1,62,228,NaN,2.5,544000.0,9303.0,14929.0,1620.0,1620.0,1017.0,146 BENTLEY STREET,10307.0,NaN,32,26,NaN,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR
1070991,1070992,5080500089,5,8050,89,NaN,JOHN GERVASI,A1,1,53,273,G,2.0,480000.0,14790.0,22975.0,14728.0,14728.0,1015.0,150 BENTLEY STREET,10307.0,NaN,24,32,NaN,NaN,NaN,NaN,1017.0,FINAL,2010/11,AC-TR
1070992,1070993,5080500092,5,8050,92,NaN,RITA M MOOG,A1,1,52,136,NaN,2.0,543000.0,12388.0,23037.0,1620.0,1620.0,1017.0,156 BENTLEY STREET,10307.0,NaN,32,21,NaN,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR
1070993,1070994,5080500094,5,8050,94,NaN,EDWARD DONOHUE,A1,1,50,80,NaN,2.0,473000.0,14061.0,22394.0,1620.0,1620.0,1017.0,162 BENTLEY STREET,10307.0,NaN,30,24,NaN,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR


In [4]:
# visualizando o shape
print(f'Quantidade de linhas:  {df.shape[0]}')
print(f'Quantidade de colunas: {df.shape[1]}')

Quantidade de linhas:  1070994
Quantidade de colunas: 32


Vamos fazer um resumo, colocando em *Data Frame*:
* **tipo_dados**: checar se cada variável foi interpretado pelo python.
* **dados_ausentes(%)**: Checar o quanto estão faltando do meu conjunto de dados
* **valores_unicos**: Olhar quantos valores únicos em cada variável, isso nos auxilia a detectar se alguma variável é categórica, por exemplo.

In [5]:
# resumo dos dados
pd.DataFrame({'tipo_dados':df.dtypes, 
              'dados_ausentes(%)':(df.isnull().mean())*100, 
              'valores_unicos':df.nunique()})

,tipo_dados,dados_ausentes(%),valores_unicos
RECORD,int64,0.000000,1070994
BBLE,object,0.000000,1070994
B,int64,0.000000,5
BLOCK,int64,0.000000,13984
LOT,int64,0.000000,6366
EASEMENT,object,99.567131,12
OWNER,object,2.964069,863346
BLDGCL,object,0.000000,200
TAXCLASS,object,0.000000,11
LTFRONT,int64,0.000000,1297


Vamos iniciar o trabalho de modelagem, mas como uma boa prática, vamos criar uma cópia do dataframe. Mas porque criar uma cópia?

Por duas razões, caso precisar retornar aos dados originais por alguma razão, não vou precisar carregar os dados novamentes.

E outra que quando calcularmos os scores, vou utilizar esse arquivo original para colocar as pontuações.

In [6]:
# criando uma cópia do dataframe
df_temp = df.copy()

## 2. Limpeza e transformação dos dados

Neste caso vamos tratar os dados ausentes, ou seja, os registros inexistentes no conjunto de dados.

Temos duas opções sobre o que fazer com esses dados:

1. Excluir a variável, só que neste caso deveria fazer caso tivessem de 70% à 80% dos dados faltando (apenas uma referência) ou se tenho certeza que não irá fazer falta.

2. Substituir esses valores, porém, precisamos fazer de forma mais fiel e próxima possível da realidade.

### 2.1. Limpando valores ausentes da variável ZIP

A variável ZIP (código postal) é uma das variáveis mais importantes, pois indica a localização dos imóveis, portanto não podemos removê-la. Aproximadamente 3% de registros faltando.

Por ser uma variável categórica, vamos optar por utilizar a moda, ou seja, o valor que mais se repete.

Vamos criar um loop em um função para obter o resultado e caso não encontrar valor da moda o resultado será 1.

In [7]:
# checando o total de valores ausentes
df['ZIP'].isna().sum()

29890

In [8]:
# criando a função para o cálculo da moda
def calcula_moda(x):
    m = pd.Series.mode(x)
    if m.empty:
        x = np.array(range(1, 11))
        return x[0]
    else:
        return m.values[0]

In [9]:
# criando uma função de contagem para calcular a frequencia do valor da moda de cada grupo (que vamos definir)
def contagem(x):
    return x.value_counts().head(1)

Nosso objetivo é encontrar um valor que chegue o mais próximo do real, para isso vamos realizar a seguinte estratégia.

Vamos agrupar duas variáveis "B" e "BLOCK" e com as funções definidas acima, vamos extrair os valores da moda para cada grupo.

In [10]:
# realizando o agrupamento com seus respectivos valores de moda
df_zip_group = df_temp.groupby(['B', 'BLOCK'])['ZIP'].agg(ZIP = calcula_moda, Count = contagem).reset_index()

# checando as primeiras linhas
df_zip_group.head(20)

,B,BLOCK,ZIP,Count
0,1,1,10004.0,2
1,1,2,10004.0,2
2,1,3,10004.0,4
3,1,4,10004.0,53
4,1,5,10004.0,9
5,1,6,10004.0,2
6,1,7,10004.0,15
7,1,8,10004.0,6
8,1,9,10004.0,4
9,1,10,10004.0,8


Agora vamos fazer da seguinte forma, vamos analisar os grupos e aqueles sem frequencia de moda, comparamos a frequencia do grupo acima e abaixo e então inserimos o valor de maior frequencia.

In [11]:
# criando o loop para o preenchimento
for i in range(len(df_zip_group)):
    if (df_zip_group.loc[i, 'ZIP'] == 1):
        if df_zip_group.loc[i - 1, 'Count'] > df_zip_group.loc[i + 1, 'Count']:
            val = df_zip_group.loc[i - 1, 'ZIP']
        else:
            val = df_zip_group.loc[i + 1, 'ZIP']
        df_zip_group.loc[i, 'ZIP'] = val
        
# verificando os primeiros registros
df_zip_group.head(20)

,B,BLOCK,ZIP,Count
0,1,1,10004.0,2
1,1,2,10004.0,2
2,1,3,10004.0,4
3,1,4,10004.0,53
4,1,5,10004.0,9
5,1,6,10004.0,2
6,1,7,10004.0,15
7,1,8,10004.0,6
8,1,9,10004.0,4
9,1,10,10004.0,8


Depois de ter encontrado os valores com maior frequencia de cada grupo formado nesse dataframe auxiliar, vamos de fato preencher os registros com valores ausentes da moda.

In [12]:
# criando a função
def preenche_moda(x):
    if pd.isnull(x['ZIP']):
        return df_zip_group[(df_zip_group['B'] == x['B']) & (df_zip_group['BLOCK'] == x['BLOCK'])]['ZIP'].values[0]
    else:
        return x['ZIP']

In [13]:
# usando a função para preencher os valores ZIP ausentes
df_temp['ZIP'] = df_temp.apply(preenche_moda, axis=1)

In [14]:
# verificando se algum valor ausente para ZIP
df_temp['ZIP'].isna().sum()

0

In [15]:
# verificando manualmente uma das ocorrências para checar se a fórmula funcionou corretamente
df_temp[(df_temp['B'] == 1) & (df_temp['BLOCK'] == 36)]['ZIP']

6092    10005.0
6093    10005.0
6094    10005.0
6095    10005.0
Name: ZIP, dtype: float64

In [16]:
# verificando os primeiros registros
df_temp.head()

,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,EXT,STORIES,FULLVAL,AVLAND,AVTOT,EXLAND,EXTOT,EXCD1,STADDR,ZIP,EXMPTCL,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,1046,NaN,NaN,21400000.0,4225500.0,9630000.0,4225500.0,9630000.0,4600.0,1 LIBERTY ISLAND,10004.0,X3,0,0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27,0,NaN,NaN,193800000.0,14310000.0,87210000.0,14310000.0,87210000.0,4600.0,1 ELLIS ISLAND,10004.0,X3,0,0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709,564,E,3.0,104686000.0,39008700.0,47108700.0,39008700.0,47108700.0,2191.0,MARGINAL STREET,10004.0,X1,709,564,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793,551,NaN,2.0,39200000.0,15255000.0,17640000.0,15255000.0,17640000.0,2191.0,PIER 6,10004.0,X1,85,551,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323,1260,NaN,1.0,272300000.0,121050000.0,122535000.0,121050000.0,122535000.0,2231.0,BATTERY PARK,10004.0,X1,89,57,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR


In [17]:
# salvando em disco
df_temp.to_csv('Dados/dataset_zip_limpo.csv')

In [18]:
# convertendo a variável ZIP, por ser categórica, em string
df_temp['ZIP'] = df_temp['ZIP'].astype(int).astype(str)

In [19]:
# criando outra cópia da estrutura do dataframe
df_temp2 = df_temp.copy()

In [20]:
df_temp2 = pd.read_csv('Dados/dataset_zip_limpo.csv')

Vamos dar sequencia limpando as variáveis numéricas quantitativas

### 2.2. Limpando valores ausentes das variáveis:

FULLVAL, AVLAND, AVTOT, BLDFRONT, BLDDEPTH, LTFRONT e LTDEPTH

In [21]:
# substituindo valores vazios por NaN, para funcionando do método fillna
df_temp2.replace(0, np.nan, inplace=True)

In [22]:
# definindo a lista das features à serem preenchidas
list_fill = ['FULLVAL', 'AVLAND', 'AVTOT', 'BLDFRONT', 'BLDDEPTH', 'LTFRONT', 'LTDEPTH']

# contando o número de NA para cada uma das features
for i in list_fill:
    print(i, "--> Número de valores ausentes", df_temp2[i].isna().sum())

FULLVAL --> Número de valores ausentes 13007
AVLAND --> Número de valores ausentes 13009
AVTOT --> Número de valores ausentes 13007
BLDFRONT --> Número de valores ausentes 228815
BLDDEPTH --> Número de valores ausentes 228853
LTFRONT --> Número de valores ausentes 169108
LTDEPTH --> Número de valores ausentes 170128


Agora que já sabemos a quantidade de valores ausentes em cada uma dessas variáveis, vamos utilizar a mediana de cada grupo para preencher esses valores.

Poderíamos ter escolhido a média, por exemplo, mas a média é influenciada por valores ausentes enquanto a mediana contorna esse problema.

Vamos pegar a mediana por grupos, porque queremos os valores mais próximo do real, neste caso grupos com tamanho maior ou igual a 5.

In [23]:
# preenchendo os registros vazios com a mediana do grupo ZIP e BLDGCL
for i in list_fill:
    df_temp2[i] = df_temp2.groupby(['ZIP', 'BLDGCL'])[i].apply(lambda x: x.fillna(x.median()) if len(x) >= 5 else x)

In [24]:
# contando o número de NA para cada uma das features
for i in list_fill:
    print(i, "--> Número de valores ausentes", df_temp2[i].isna().sum())

FULLVAL --> Número de valores ausentes 10294
AVLAND --> Número de valores ausentes 10294
AVTOT --> Número de valores ausentes 10294
BLDFRONT --> Número de valores ausentes 75202
BLDDEPTH --> Número de valores ausentes 75218
LTFRONT --> Número de valores ausentes 16025
LTDEPTH --> Número de valores ausentes 17764


In [25]:
# preenchendo os registros vazios com a mediana do grupo ZIP e TAXCLASS
for i in list_fill:
    df_temp2[i] = df_temp2.groupby(['ZIP', 'TAXCLASS'])[i].apply(lambda x: x.fillna(x.median()) if len(x) >= 5 else x)
    
# contando o número de NA para cada uma das features
for i in list_fill:
    print(i, "--> Número de valores ausentes", df_temp2[i].isna().sum())

FULLVAL --> Número de valores ausentes 3915
AVLAND --> Número de valores ausentes 3915
AVTOT --> Número de valores ausentes 3915
BLDFRONT --> Número de valores ausentes 31226
BLDDEPTH --> Número de valores ausentes 31130
LTFRONT --> Número de valores ausentes 3092
LTDEPTH --> Número de valores ausentes 3146


In [26]:
# preenchendo os registros vazios com a mediana do grupo B e TAXCLASS
for i in list_fill:
    df_temp2[i] = df_temp2.groupby(['B', 'TAXCLASS'])[i].apply(lambda x: x.fillna(x.median()) if len(x) >= 5 else x)
    
# contando o número de NA para cada uma das features
for i in list_fill:
    print(i, "--> Número de valores ausentes", df_temp2[i].isna().sum())

FULLVAL --> Número de valores ausentes 678
AVLAND --> Número de valores ausentes 678
AVTOT --> Número de valores ausentes 678
BLDFRONT --> Número de valores ausentes 22317
BLDDEPTH --> Número de valores ausentes 18831
LTFRONT --> Número de valores ausentes 2
LTDEPTH --> Número de valores ausentes 2


In [27]:
# preenchendo os registros vazios com a mediana do grupo B
for i in list_fill:
    df_temp2[i] = df_temp2.groupby(['B'])[i].apply(lambda x: x.fillna(x.median()) if len(x) >= 5 else x)
    
# contando o número de NA para cada uma das features
for i in list_fill:
    print(i, "--> Número de valores ausentes", df_temp2[i].isna().sum())

FULLVAL --> Número de valores ausentes 0
AVLAND --> Número de valores ausentes 0
AVTOT --> Número de valores ausentes 0
BLDFRONT --> Número de valores ausentes 0
BLDDEPTH --> Número de valores ausentes 0
LTFRONT --> Número de valores ausentes 0
LTDEPTH --> Número de valores ausentes 0


### 2.3. Limpando valores ausentes da variável STORIES

Como já vimos, essa variável representa o número de andares. Também vamos utilizar a mediana para substituição, primeiro agrupando por duas variáveis e depois utilizando a própria variável.

In [28]:
# Checando valores ausentes
print('STORIES', "--> Número de valores ausentes:", df_temp2['STORIES'].isna().sum())

STORIES --> Número de valores ausentes: 56264


In [29]:
# imputando a mediana com base nos grupos de ZIP e BLDGCL
df_temp2['STORIES'] = df_temp2.groupby(['ZIP', 'BLDGCL'])['STORIES'].apply(lambda x: x.fillna(x.median()) if len(x) >= 5 else x)

# Checando valores ausentes
print('STORIES', "--> Número de valores ausentes:", df_temp2['STORIES'].isna().sum())

STORIES --> Número de valores ausentes: 42102


In [30]:
# imputando a mediana com base nos grupos de ZIP e BLDGCL
df_temp2['STORIES'] = df_temp2.groupby(['BLDGCL'])['STORIES'].apply(lambda x: x.fillna(x.median()) if len(x) >= 5 else x)

# Checando valores ausentes
print('STORIES', "--> Número de valores ausentes:", df_temp2['STORIES'].isna().sum())

STORIES --> Número de valores ausentes: 4280


In [31]:
# imputando a mediana com base nos grupos de ZIP e BLDGCL
df_temp2['STORIES'] = df_temp2.groupby(['TAXCLASS'])['STORIES'].apply(lambda x: x.fillna(x.median()) if len(x) >= 5 else x)

# Checando valores ausentes
print('STORIES', "--> Número de valores ausentes:", df_temp2['STORIES'].isna().sum())

STORIES --> Número de valores ausentes: 0


In [32]:
df_temp2.to_csv('Dados/dataset_variaveis_limpo.csv')

## 3. Engenharia de Atributos

Vamos criar algumas variáveis a partir de operações com as variáveis existentes, mas por que?

A Engenharia de Atributos ou Features Engineering é quando criados novas variáveis a partir das que já existem, como por exemplo, vamos criar a variável Área, pensando no cálculo da área podemos multiplicar a variavel LTFRONT pela variável LTDEPTH.

Relembrando essas duas variáveis representam, respectivamente, a frente do lote e a profundidade, com isso base * altura, teremos a área total calculada.

Mas antes, vamos fazer uma cópia do dataset original como boa prática, caso algo der errado.

In [33]:
# copiando o dataframe
df_proc = df_temp2.copy()

In [34]:
# calculando a área LTFRONT * LTDEPHT
df_proc['AREA1'] = df_proc['LTFRONT'] * df_proc['LTDEPTH']

# visualizando esse resultado
df_proc[['LTFRONT', 'LTDEPTH', 'AREA1']].head()

,LTFRONT,LTDEPTH,AREA1
0,500.0,1046.0,523000.0
1,27.0,355.0,9585.0
2,709.0,564.0,399876.0
3,793.0,551.0,436943.0
4,323.0,1260.0,406980.0


Com qual objetivo é fazer esse tipo de trabalho?

Com a variável AREA1 criada agora, posso remover do modelo as outras duas variáveis (LTFRONT e LTDEPTH), pois uma variável representa essas duas agora, com isso reduzimos o número de variáveis do modelo, no qual facilita para os padrões serem encontrados por ele, além do ganho de performance no processamento.

Seguindo, vamos criar outra variável AREA2 no qual vamos multiplicar BLDFRONT por BLDDEPTH.

In [35]:
# calculando a área LTFRONT * LTDEPHT
df_proc['AREA2'] = df_proc['BLDFRONT'] * df_proc['BLDDEPTH']

# visualizando esse resultado
df_proc[['BLDFRONT', 'BLDDEPTH', 'AREA2']].head()

,BLDFRONT,BLDDEPTH,AREA2
0,382.0,318.0,121476.0
1,382.0,318.0,121476.0
2,709.0,564.0,399876.0
3,85.0,551.0,46835.0
4,89.0,57.0,5073.0


E então por, vamos calcular mais uma área, dessa vez multiplicando a AREA2 pela STORIES.

In [36]:
# calculando a área LTFRONT * LTDEPHT
df_proc['AREA3'] = df_proc['AREA2'] * df_proc['STORIES']

# visualizando esse resultado
df_proc[['AREA2', 'STORIES', 'AREA3']].head()

,AREA2,STORIES,AREA3
0,121476.0,4.0,485904.0
1,121476.0,1.0,121476.0
2,399876.0,3.0,1199628.0
3,46835.0,2.0,93670.0
4,5073.0,1.0,5073.0


É obrigatório criar essas variáveis, a resposta é não! Vai depender muito do problema que você quer resolver, precisamos ter em mente o como vamos "facilitar" para o algoritmo encontrar os padrões e que tenha o maior número de acertos possíveis, desde que fique generalizado.

Podemos ir testando essas variáveis mantendo o algoritmo menos complexo também e no final decidimos se manteremos e quais vamos manter, algumas ou todas.

Mas não paramos por ai, vamos agora criar várias colunas com índices utilizando essas variáveis recém criadas, utilizando FULLVAL (valor do imóvel), AVLAND (valor do terreno) e AVTOT (Valor total do imóvel) com as AREAS criadas.

In [37]:
# criando um loop para criação dessas variáveis
k = 1
for i in ['FULLVAL', 'AVLAND', 'AVTOT']:
    for j in ['AREA1', 'AREA2', 'AREA3']:
        indice_area = 'ind' + str(k)
        df_proc[indice_area] = df_proc[i] / df_proc[j]
        print(indice_area + ' é a combinação feira entre:', i, j)
        k += 1

ind1 é a combinação feira entre: FULLVAL AREA1
ind2 é a combinação feira entre: FULLVAL AREA2
ind3 é a combinação feira entre: FULLVAL AREA3
ind4 é a combinação feira entre: AVLAND AREA1
ind5 é a combinação feira entre: AVLAND AREA2
ind6 é a combinação feira entre: AVLAND AREA3
ind7 é a combinação feira entre: AVTOT AREA1
ind8 é a combinação feira entre: AVTOT AREA2
ind9 é a combinação feira entre: AVTOT AREA3


In [38]:
# vizualizando uma amostra do resultado
df_proc.head()

,Unnamed: 0,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,EXT,STORIES,FULLVAL,AVLAND,AVTOT,EXLAND,EXTOT,EXCD1,STADDR,ZIP,EXMPTCL,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE,AREA1,AREA2,AREA3,ind1,ind2,ind3,ind4,ind5,ind6,ind7,ind8,ind9
0,NaN,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500.0,1046.0,NaN,4.0,21400000.0,4225500.0,9630000.0,4225500.0,9630000.0,4600.0,1 LIBERTY ISLAND,10004.0,X3,382.0,318.0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR,523000.0,121476.0,485904.0,40.917782,176.166486,44.041621,8.079350,34.784649,8.696162,18.413002,79.274919,19.818730
1,1.0,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27.0,355.0,NaN,1.0,193800000.0,14310000.0,87210000.0,14310000.0,87210000.0,4600.0,1 ELLIS ISLAND,10004.0,X3,382.0,318.0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR,9585.0,121476.0,121476.0,20219.092332,1595.376865,1595.376865,1492.957746,117.801047,117.801047,9098.591549,717.919589,717.919589
2,2.0,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709.0,564.0,E,3.0,104686000.0,39008700.0,47108700.0,39008700.0,47108700.0,2191.0,MARGINAL STREET,10004.0,X1,709.0,564.0,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR,399876.0,399876.0,1199628.0,261.796157,261.796157,87.265386,97.551991,97.551991,32.517330,117.808271,117.808271,39.269424
3,3.0,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793.0,551.0,NaN,2.0,39200000.0,15255000.0,17640000.0,15255000.0,17640000.0,2191.0,PIER 6,10004.0,X1,85.0,551.0,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR,436943.0,46835.0,93670.0,89.714219,836.980890,418.490445,34.913021,325.717946,162.858973,40.371399,376.641401,188.320700
4,4.0,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323.0,1260.0,NaN,1.0,272300000.0,121050000.0,122535000.0,121050000.0,122535000.0,2231.0,BATTERY PARK,10004.0,X1,89.0,57.0,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR,406980.0,5073.0,5073.0,669.074647,53676.325646,53676.325646,297.434763,23861.620343,23861.620343,301.083591,24154.346541,24154.346541


Reforçando, criamos essas variáveis a mais para que ajude o algoritmo a detectar os **padrões existentes**, importante saber que não criamos padrões, apenas "facilitamos" quando possível o algoritmo detectá-los, se existirem.

Vamos ainda criar mais algumas?
Vamos utilizar as médias dos índices agrupando por outras classes, isso faz com que olhemos os padrões de outra perspectiva que irá ajudar o modelo mais a frente.

Tudo isso são técnicas de feature engineering, com base na junção da informação de variáveis do conjunto de dados.

A princípio vamos converter a variável *ZIP* de "float64" para "string".

In [39]:
# checando o tipo da variável
df_proc['ZIP'].dtype

dtype('float64')

In [40]:
# convertendo a variável
df_proc['ZIP'] = df_proc.ZIP.astype(int).astype(str)

# checando o tipo da variável
df_proc['ZIP'].dtype

dtype('O')

Agora, vamos criar uma lista com 3 variáveis, *ZIP*, *TAXCLASS* e *B*, respectivamente são, código postal, classe do imposto e o bairro.

Selecionamos essas três variáveis porque de acordo com nossa análise exploratória, são variáveis que parecem ser relevantes para o problema de negócio, e esse tipo de informação poderíamos validar com a área de negócio também.

In [41]:
# criando uma lista das variáveis
list_groupby = ['ZIP', 'TAXCLASS', 'B']

Vamos criar um loop no qual ele fará o agrupamento pelo indices e depois o cálculo da média.

In [42]:
# criando variáveis que representam as médias das áreas dos imóveis por grupo com base no índice criado

# indice da lista
ind_lista = ['ind' + str(i) for i in range(1,10)]

# loop
for i in list_groupby:
    for j in ind_lista:
        name_col = 'media_' + str(j) + '_grupo_' + str(i)
        df_proc[name_col] = df_proc.groupby(i)[j].transform('mean')
        name_col_final = str(j) + '_' + name_col
        df_proc[name_col_final] = df_proc[j] / df_proc[name_col]

In [43]:
# visualizando as primeiras linhas
df_proc.head()

,Unnamed: 0,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,EXT,STORIES,FULLVAL,AVLAND,AVTOT,EXLAND,EXTOT,EXCD1,STADDR,ZIP,EXMPTCL,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE,AREA1,AREA2,AREA3,ind1,ind2,ind3,ind4,ind5,ind6,ind7,ind8,ind9,media_ind1_grupo_ZIP,ind1_media_ind1_grupo_ZIP,media_ind2_grupo_ZIP,ind2_media_ind2_grupo_ZIP,media_ind3_grupo_ZIP,ind3_media_ind3_grupo_ZIP,media_ind4_grupo_ZIP,ind4_media_ind4_grupo_ZIP,media_ind5_grupo_ZIP,ind5_media_ind5_grupo_ZIP,media_ind6_grupo_ZIP,ind6_media_ind6_grupo_ZIP,media_ind7_grupo_ZIP,ind7_media_ind7_grupo_ZIP,media_ind8_grupo_ZIP,ind8_media_ind8_grupo_ZIP,media_ind9_grupo_ZIP,ind9_media_ind9_grupo_ZIP,media_ind1_grupo_TAXCLASS,ind1_media_ind1_grupo_TAXCLASS,media_ind2_grupo_TAXCLASS,ind2_media_ind2_grupo_TAXCLASS,media_ind3_grupo_TAXCLASS,ind3_media_ind3_grupo_TAXCLASS,media_ind4_grupo_TAXCLASS,ind4_media_ind4_grupo_TAXCLASS,media_ind5_grupo_TAXCLASS,ind5_media_ind5_grupo_TAXCLASS,media_ind6_grupo_TAXCLASS,ind6_media_ind6_grupo_TAXCLASS,media_ind7_grupo_TAXCLASS,ind7_media_ind7_grupo_TAXCLASS,media_ind8_grupo_TAXCLASS,ind8_media_ind8_grupo_TAXCLASS,media_ind9_grupo_TAXCLASS,ind9_media_ind9_grupo_TAXCLASS,media_ind1_grupo_B,ind1_media_ind1_grupo_B,media_ind2_grupo_B,ind2_media_ind2_grupo_B,media_ind3_grupo_B,ind3_media_ind3_grupo_B,media_ind4_grupo_B,ind4_media_ind4_grupo_B,media_ind5_grupo_B,ind5_media_ind5_grupo_B,media_ind6_grupo_B,ind6_media_ind6_grupo_B,media_ind7_grupo_B,ind7_media_ind7_grupo_B,media_ind8_grupo_B,ind8_media_ind8_grupo_B,media_ind9_grupo_B,ind9_media_ind9_grupo_B
0,NaN,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500.0,1046.0,NaN,4.0,21400000.0,4225500.0,9630000.0,4225500.0,9630000.0,4600.0,1 LIBERTY ISLAND,10004,X3,382.0,318.0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR,523000.0,121476.0,485904.0,40.917782,176.166486,44.041621,8.079350,34.784649,8.696162,18.413002,79.274919,19.818730,308.567411,0.132606,399.54316,0.440920,136.599794,0.322414,43.91237,0.183988,90.426622,0.384673,52.264394,0.166388,126.01684,0.146115,166.965375,0.474799,58.52076,0.338662,219.170893,0.186694,861.398717,0.204512,449.71508,0.097932,39.657134,0.203730,274.576789,0.126685,163.840149,0.053077,98.698597,0.186558,408.041766,0.194281,209.16395,0.094752,363.950199,0.112427,554.922709,0.317461,162.784056,0.270552,35.93064,0.224860,67.911275,0.512207,31.906732,0.272549,95.545234,0.192715,146.188725,0.542278,45.061308,0.439817
1,1.0,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27.0,355.0,NaN,1.0,193800000.0,14310000.0,87210000.0,14310000.0,87210000.0,4600.0,1 ELLIS ISLAND,10004,X3,382.0,318.0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR,9585.0,121476.0,121476.0,20219.092332,1595.376865,1595.376865,1492.957746,117.801047,117.801047,9098.591549,717.919589,717.919589,308.567411,65.525689,399.54316,3.993003,136.599794,11.679204,43.91237,33.998569,90.426622,1.302725,52.264394,2.253945,126.01684,72.201394,166.965375,4.299811,58.52076,12.267776,219.170893,92.252635,861.398717,1.852077,449.71508,3.547528,39.657134,37.646638,274.576789,0.429028,163.840149,0.719000,98.698597,92.185622,408.041766,1.759427,209.16395,3.432329,363.950199,55.554558,554.922709,2.874953,162.784056,9.800572,35.93064,41.551103,67.911275,1.734632,31.906732,3.692044,95.545234,95.228104,146.188725,4.910909,45.061308,15.932063
2,2.0,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709.0,564.0,E,3.0,104686000.0,39008700.0,47108700.0,39008700.0,47108700.0,2191.0,MARGINAL STREET,10004,X1,709.0,564.0,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR,399876.0,399876.0,1199628.0,261.796157,261.796157,87.265386,97.551991,97.551991,32.517330,117.808271,117.808271,39.269424,308.567411,0.848425,399.54316,0.655239,136.599794,0.638840,43.91237,2.221515,90.426622,1.078797,52.264394,0.622170,126.01684,0.934861,166.965375,0.705585,58.52076,0.671034,219.170893,1.194484,861.398717,0.303920,449.71508,0.194046,39.657

Qual a explicação para criar essas médias?<br>
Pode ser que a média dos grupos seja mais relevante que os índices em si para o algoritmo e caso não for satisfatório voltamos para esta etapa de feature engineering e faremos de outra forma.

In [44]:
# checando as dimensões
df_proc.shape

(1070994, 99)

Depois calcular a média da agrupamento daquelas 3 variáveis pelos índices, vamos também calcular a média para todas as demais variáveis.

In [45]:
# criando um loop para todas as variáveis
for j in ind_lista:
    name_col = 'media_' + str(j) + '_grupo_ALL'
    df_proc[name_col] = df_proc[j].mean()
    name_col_final = str(j) + '_' + name_col
    df_proc[name_col_final] = df_proc[j] / df_proc[name_col]

# visualizando as primeiras linhas
df_proc.head()

,Unnamed: 0,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,EXT,STORIES,FULLVAL,AVLAND,AVTOT,EXLAND,EXTOT,EXCD1,STADDR,ZIP,EXMPTCL,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE,AREA1,AREA2,AREA3,ind1,ind2,ind3,ind4,ind5,ind6,ind7,ind8,ind9,media_ind1_grupo_ZIP,ind1_media_ind1_grupo_ZIP,media_ind2_grupo_ZIP,ind2_media_ind2_grupo_ZIP,media_ind3_grupo_ZIP,ind3_media_ind3_grupo_ZIP,media_ind4_grupo_ZIP,ind4_media_ind4_grupo_ZIP,media_ind5_grupo_ZIP,ind5_media_ind5_grupo_ZIP,media_ind6_grupo_ZIP,ind6_media_ind6_grupo_ZIP,media_ind7_grupo_ZIP,ind7_media_ind7_grupo_ZIP,media_ind8_grupo_ZIP,ind8_media_ind8_grupo_ZIP,media_ind9_grupo_ZIP,ind9_media_ind9_grupo_ZIP,media_ind1_grupo_TAXCLASS,ind1_media_ind1_grupo_TAXCLASS,media_ind2_grupo_TAXCLASS,ind2_media_ind2_grupo_TAXCLASS,media_ind3_grupo_TAXCLASS,ind3_media_ind3_grupo_TAXCLASS,media_ind4_grupo_TAXCLASS,ind4_media_ind4_grupo_TAXCLASS,media_ind5_grupo_TAXCLASS,ind5_media_ind5_grupo_TAXCLASS,media_ind6_grupo_TAXCLASS,ind6_media_ind6_grupo_TAXCLASS,media_ind7_grupo_TAXCLASS,ind7_media_ind7_grupo_TAXCLASS,media_ind8_grupo_TAXCLASS,ind8_media_ind8_grupo_TAXCLASS,media_ind9_grupo_TAXCLASS,ind9_media_ind9_grupo_TAXCLASS,media_ind1_grupo_B,ind1_media_ind1_grupo_B,media_ind2_grupo_B,ind2_media_ind2_grupo_B,media_ind3_grupo_B,ind3_media_ind3_grupo_B,media_ind4_grupo_B,ind4_media_ind4_grupo_B,media_ind5_grupo_B,ind5_media_ind5_grupo_B,media_ind6_grupo_B,ind6_media_ind6_grupo_B,media_ind7_grupo_B,ind7_media_ind7_grupo_B,media_ind8_grupo_B,ind8_media_ind8_grupo_B,media_ind9_grupo_B,ind9_media_ind9_grupo_B,media_ind1_grupo_ALL,ind1_media_ind1_grupo_ALL,media_ind2_grupo_ALL,ind2_media_ind2_grupo_ALL,media_ind3_grupo_ALL,ind3_media_ind3_grupo_ALL,media_ind4_grupo_ALL,ind4_media_ind4_grupo_ALL,media_ind5_grupo_ALL,ind5_media_ind5_grupo_ALL,media_ind6_grupo_ALL,ind6_media_ind6_grupo_ALL,media_ind7_grupo_ALL,ind7_media_ind7_grupo_ALL,media_ind8_grupo_ALL,ind8_media_ind8_grupo_ALL,media_ind9_grupo_ALL,ind9_media_ind9_grupo_ALL
0,NaN,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500.0,1046.0,NaN,4.0,21400000.0,4225500.0,9630000.0,4225500.0,9630000.0,4600.0,1 LIBERTY ISLAND,10004,X3,382.0,318.0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR,523000.0,121476.0,485904.0,40.917782,176.166486,44.041621,8.079350,34.784649,8.696162,18.413002,79.274919,19.818730,308.567411,0.132606,399.54316,0.440920,136.599794,0.322414,43.91237,0.183988,90.426622,0.384673,52.264394,0.166388,126.01684,0.146115,166.965375,0.474799,58.52076,0.338662,219.170893,0.186694,861.398717,0.204512,449.71508,0.097932,39.657134,0.203730,274.576789,0.126685,163.840149,0.053077,98.698597,0.186558,408.041766,0.194281,209.16395,0.094752,363.950199,0.112427,554.922709,0.317461,162.784056,0.270552,35.93064,0.224860,67.911275,0.512207,31.906732,0.272549,95.545234,0.192715,146.188725,0.542278,45.061308,0.439817,343.735061,0.119039,607.276406,0.290093,283.67069,0.155256,11.094569,0.728226,44.11249,0.788544,23.912472,0.363666,30.658337,0.600587,75.635641,1.048116,34.430619,0.575614
1,1.0,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27.0,355.0,NaN,1.0,193800000.0,14310000.0,87210000.0,14310000.0,87210000.0,4600.0,1 ELLIS ISLAND,10004,X3,382.0,318.0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR,9585.0,121476.0,121476.0,20219.092332,1595.376865,1595.376865,1492.957746,117.801047,117.801047,9098.591549,717.919589,717.919589,308.567411,65.525689,399.54316,3.993003,136.599794,11.679204,43.91237,33.998569,90.426622,1.302725,52.264394,2.253945,126.01684,72.201394,166.965375,4.299811,58.52076,12.267776,219.170893,92.252635,861.398717,1.852077,449.71508,3.547528,39.657134,37.646638,274.576789,0.429028,163.840149,0.719000,98.698597,92.185622,408.041766,1.759427,209.16395,3.432329,363.950199,55.554558,554.922709,2.874953,162.784056,9.800572,35.93064,41.551103,67.911275,1.734632,31.906732,3.692044,95.545234,95.228104,146.188725,4.910909,45.061308,15.932063,343.73

In [46]:
# checando as dimensões
df_proc.shape

(1070994, 117)

In [47]:
# criando uma lista vazia para coletar as colunas criadas com a média
cols_created = []

# loop pelas variáveis do group by
for i in list_groupby:
    for j in ind_lista:
        cols_created.append(j + '_media_' + j + '_grupo_' + i)
        
# checando o tamanho dessa nova lista
len(cols_created)

27

In [48]:
# loop para todas as colunas
for j in ind_lista:
    cols_created.append(j + '_media_' + j + '_grupo_ALL')
    
# checando o tamanho dessa nova lista
len(cols_created)

36

Então agora vamos inserir no conjunto de dados original.

In [49]:
# concatenando todas as colunas em uma lista para o conjunto final
cols = list(df.columns) + cols_created

# criando o dataframe final
df_final = df_proc[cols]

# visualizando a dimensão
df_final.shape

(1070994, 68)

In [50]:
# verificando todas as variáveis
df_final.columns

Index(['RECORD', 'BBLE', 'B', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
       'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND',
       'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL',
       'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2',
       'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE', 'ind1_media_ind1_grupo_ZIP',
       'ind2_media_ind2_grupo_ZIP', 'ind3_media_ind3_grupo_ZIP',
       'ind4_media_ind4_grupo_ZIP', 'ind5_media_ind5_grupo_ZIP',
       'ind6_media_ind6_grupo_ZIP', 'ind7_media_ind7_grupo_ZIP',
       'ind8_media_ind8_grupo_ZIP', 'ind9_media_ind9_grupo_ZIP',
       'ind1_media_ind1_grupo_TAXCLASS', 'ind2_media_ind2_grupo_TAXCLASS',
       'ind3_media_ind3_grupo_TAXCLASS', 'ind4_media_ind4_grupo_TAXCLASS',
       'ind5_media_ind5_grupo_TAXCLASS', 'ind6_media_ind6_grupo_TAXCLASS',
       'ind7_media_ind7_grupo_TAXCLASS', 'ind8_media_ind8_grupo_TAXCLASS',
       'ind9_media_ind9_grupo_TAXCLASS', 'ind1_media_ind1_grupo_

In [51]:
# visualizando as primeiras linhas
df_final.head()

,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,EXT,STORIES,FULLVAL,AVLAND,AVTOT,EXLAND,EXTOT,EXCD1,STADDR,ZIP,EXMPTCL,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE,ind1_media_ind1_grupo_ZIP,ind2_media_ind2_grupo_ZIP,ind3_media_ind3_grupo_ZIP,ind4_media_ind4_grupo_ZIP,ind5_media_ind5_grupo_ZIP,ind6_media_ind6_grupo_ZIP,ind7_media_ind7_grupo_ZIP,ind8_media_ind8_grupo_ZIP,ind9_media_ind9_grupo_ZIP,ind1_media_ind1_grupo_TAXCLASS,ind2_media_ind2_grupo_TAXCLASS,ind3_media_ind3_grupo_TAXCLASS,ind4_media_ind4_grupo_TAXCLASS,ind5_media_ind5_grupo_TAXCLASS,ind6_media_ind6_grupo_TAXCLASS,ind7_media_ind7_grupo_TAXCLASS,ind8_media_ind8_grupo_TAXCLASS,ind9_media_ind9_grupo_TAXCLASS,ind1_media_ind1_grupo_B,ind2_media_ind2_grupo_B,ind3_media_ind3_grupo_B,ind4_media_ind4_grupo_B,ind5_media_ind5_grupo_B,ind6_media_ind6_grupo_B,ind7_media_ind7_grupo_B,ind8_media_ind8_grupo_B,ind9_media_ind9_grupo_B,ind1_media_ind1_grupo_ALL,ind2_media_ind2_grupo_ALL,ind3_media_ind3_grupo_ALL,ind4_media_ind4_grupo_ALL,ind5_media_ind5_grupo_ALL,ind6_media_ind6_grupo_ALL,ind7_media_ind7_grupo_ALL,ind8_media_ind8_grupo_ALL,ind9_media_ind9_grupo_ALL
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500.0,1046.0,NaN,4.0,21400000.0,4225500.0,9630000.0,4225500.0,9630000.0,4600.0,1 LIBERTY ISLAND,10004,X3,382.0,318.0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR,0.132606,0.440920,0.322414,0.183988,0.384673,0.166388,0.146115,0.474799,0.338662,0.186694,0.204512,0.097932,0.203730,0.126685,0.053077,0.186558,0.194281,0.094752,0.112427,0.317461,0.270552,0.224860,0.512207,0.272549,0.192715,0.542278,0.439817,0.119039,0.290093,0.155256,0.728226,0.788544,0.363666,0.600587,1.048116,0.575614
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27.0,355.0,NaN,1.0,193800000.0,14310000.0,87210000.0,14310000.0,87210000.0,4600.0,1 ELLIS ISLAND,10004,X3,382.0,318.0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR,65.525689,3.993003,11.679204,33.998569,1.302725,2.253945,72.201394,4.299811,12.267776,92.252635,1.852077,3.547528,37.646638,0.429028,0.719000,92.185622,1.759427,3.432329,55.554558,2.874953,9.800572,41.551103,1.734632,3.692044,95.228104,4.910909,15.932063,58.821734,2.627102,5.624045,134.566543,2.670469,4.926343,296.773816,9.491816,20.851196
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709.0,564.0,E,3.0,104686000.0,39008700.0,47108700.0,39008700.0,47108700.0,2191.0,MARGINAL STREET,10004,X1,709.0,564.0,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR,0.848425,0.655239,0.638840,2.221515,1.078797,0.622170,0.934861,0.705585,0.671034,1.194484,0.303920,0.194046,2.459885,0.355281,0.198470,1.193616,0.288716,0.187745,0.719319,0.471770,0.536081,2.715008,1.436462,1.019137,1.233010,0.805864,0.871467,0.761622,0.431099,0.307629,8.792770,2.211437,1.359848,3.842618,1.557576,1.140538
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793.0,551.0,NaN,2.0,39200000.0,15255000.0,17640000.0,15255000.0,17640000.0,2191.0,PIER 6,10004,X1,85.0,551.0,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR,0.290744,2.094845,3.063624,0.795061,3.602014,3.116060,0.320365,2.255805,3.218015,0.409335,0.971653,0.930568,0.880372,1.186254,0.994011,0.409037,0.923046,0.900350,0.246501,1.508284,2.570832,0.971678,4.796228,5.104220,0.422537,2.576405,4.179211,0.260998,1.378254,1.475269,3.146857,7.383803,6.810629,1.316816,4.979681,5.469571
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323.0,1260.0,NaN,1.0,272300000.0,121050000.0,122535000.0,121050000.0,122535000.0,2231.0,BATTERY PARK,10004,X1,89.0,57.0,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR,2.168326,134.344249,392.945875,6.773371,263.878267,456.555954,2.389233,144.666800,412.748342,3.052753,62.312985,119.356295,7.500158,86.903268,145.639640,3.050536,59.195770,115.480448,1.838369,96.727571,329.739454,8.278026,351.364636,747.855349,3.151215,165.227152,536.032968,1.946484,88.388624,189.220556,26.809043,540.9

In [52]:
# salvando em disco
df_final.to_csv('Dados/dataset_final.csv')

## 4. Conclusão

Conseguimos preencher todas os dados faltantes e chegamos a ter 117 features, que conseguimos reduzir para 68. Ufa! Agora estamos prontos para a etapa final de modelagem e calculo do Score de Fraude.